In [1]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [30]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from __future__ import annotations
import torch
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from collections import OrderedDict
from typing import List
from functools import partial
from glasses.nn.blocks import ConvBnAct, Conv2dPad
from glasses.nn.models.segmentation.unet import UnetEncoder, Unet


In [51]:
from glasses.nn.models.classification import ResNet

resnet34 = ResNet.resnet34()
resnet34.encoder.blocks[0].block[0].block.block.conv1 = Conv2dPad(1, 64, kernel_size=1)

unet = Unet(1, 2, blocks_sizes=resnet34.encoder.blocks_sizes)
x = torch.rand((1, 1, 32 * 12, 32*12))


unet.encoder = resnet34.encoder

unet
unet(x).shape

# unet.decoder

torch.Size([1, 2, 384, 384])

In [31]:
[r.shape for r in unet.residuals]

[torch.Size([1, 1024, 25, 25]),
 torch.Size([1, 512, 50, 50]),
 torch.Size([1, 256, 100, 100]),
 torch.Size([1, 128, 200, 200]),
 torch.Size([1, 64, 400, 400])]

In [34]:
from glasses.nn.models.classification.mobilenet import InvertedResidualBlock
from glasses.nn.models.classification.se import SEModule


In [11]:
UpBlock(32, 64)

UpBlock(
  (up): ConvTranspose2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
  (block): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2dPad(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
      )
      (1): ConvBnAct(
        (conv): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
      )
    )
  )
)